In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


In [49]:
import itertools

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
582,tt0250224,13000000,16929123,Crazy/Beautiful,Kirsten Dunst|Jay Hernandez|Bruce Davison|Herm...,John Stockwell,When it's real. When it's right. Don't let any...,"At Pacific Palisades High, a poor Latino falls...",99,Drama|Romance,Touchstone Pictures|Ufland,6/29/2001,6.6,2001
1227,tt1939659,30000000,82394288,Carrie,ChloÃ« Grace Moretz|Julianne Moore|Judy Greer|...,Kimberly Peirce,Know her name. Fear her power.,A reimagining of the classic horror tale about...,100,Drama|Horror,Screen Gems|Metro-Goldwyn-Mayer (MGM)|Misher F...,10/16/2013,5.8,2013
456,tt1423995,22000000,9479718,Stone,Frances Conroy|Robert De Niro|Edward Norton|Mi...,John Curran,Some People Tell Lies. Others Live Them.,Parole officer Jack Mabry has only a few weeks...,105,Drama,Stone Productions|Mimran Schur Pictures,10/7/2010,4.9,2010
348,tt1225822,8000000,10848783,Extract,Jason Bateman|Mila Kunis|Kristen Wiig|Ben Affl...,Mike Judge,Sticking it to the man has never looked so good.,The owner of a factory that produces flavor ex...,92,Comedy,Miramax Films,9/3/2009,5.8,2009
1727,tt0465602,39000000,26820641,Shoot 'Em Up,Clive Owen|Monica Bellucci|Paul Giamatti|Steph...,Michael Davis,Just another family man making a living.,A man named Mr. Smith delivers a woman's baby ...,86,Action|Thriller|Comedy|Crime,New Line Cinema|Angry Films,7/26/2007,6.2,2007


In [0]:
data.describe()

# Предобработка

In [9]:
#Создаём словарь ответов
answers = {} 


In [370]:
data.axes

[RangeIndex(start=0, stop=1889, step=1),
 Index(['imdb_id', 'budget', 'revenue', 'original_title', 'cast', 'director',
        'tagline', 'overview', 'runtime', 'genres', 'production_companies',
        'release_date', 'vote_average', 'release_year', 'profit', 'timestamp',
        'month', 'ones', 'tagline_length', 'isprofit', 'castlist', 'length',
        'words', 'words_counted'],
       dtype='object')]

In [381]:
#создадим колонку для прибыли от каждого фильма
profit=pd.Series(data.revenue-data.budget,name='profit')
data['profit']=profit

In [200]:
#создадим колонку, в которой наличие или отсутствие прибыли будет кодироваться единицей или нулём
profit[profit <= 0] = 0
profit[profit > 0] = 1
data['isprofit'] = pd.Series(profit,name='isprofit')

In [274]:
#создадим колонку, в которой список актёров, снимавшихся в фильме - это list 
data['castlist']=data.cast.str.split('|')

In [382]:
type(data.castlist[0])

list

In [197]:
profit

0       1
1       1
2       1
3       1
4       1
       ..
1884    1
1885    0
1886    0
1887    0
1888    0
Name: profit, Length: 1889, dtype: int64

In [5]:
#создадим колонку отпечатка времени
data['timestamp']=pd.to_datetime(data.release_date)

In [6]:
#из предыдущей колонки получим колонку с месяцем выпуска фильма
data['month'] = pd.DatetimeIndex(data.timestamp).month


In [7]:
#создадим колонку с единицами, которую удобно использовать для расчётов
data['ones']=1

In [389]:
#извлечём жанры в отдельный датафрейм в формате one-hot encoding
genres_ds = data.genres.str.split('|', expand=True)
genres_extracted = pd.DataFrame(genres_ds.values.tolist())
genres_extracted = genres_extracted.stack().str.get_dummies().sum(level=0)

In [390]:
genres_extracted

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
3,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1885,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
1886,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1887,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [400]:
#извлечём актёрский состав в отдельный датафрейм
cast_extracted=data.cast.str.split('|',expand=True)

In [401]:
cast_extracted

,0,1,2,3,4
0,Chris Pratt,Bryce Dallas Howard,Irrfan Khan,Vincent D'Onofrio,Nick Robinson
1,Tom Hardy,Charlize Theron,Hugh Keays-Byrne,Nicholas Hoult,Josh Helman
2,Shailene Woodley,Theo James,Kate Winslet,Ansel Elgort,Miles Teller
3,Harrison Ford,Mark Hamill,Carrie Fisher,Adam Driver,Daisy Ridley
4,Vin Diesel,Paul Walker,Jason Statham,Michelle Rodriguez,Dwayne Johnson
...,...,...,...,...,...
1884,Patrick Stewart,Hugh Jackman,Ian McKellen,Halle Berry,Famke Janssen
1885,Richard E. Grant,Jonathan Lipnicki,Jim Carter,Alice Krige,Pamela Gidley
1886,Rene Russo,Jason Alexander,Piper Perabo,Randy Quaid,Robert De Niro
1887,Meg Ryan,Diane Keaton,Lisa Kudrow,Walter Matthau,Adam Arkin


In [426]:
#создадим список компаний в формате list
data['companies_list'] = data.production_companies.str.split('|')
                     

In [ ]:
#создадим колонку с длинами названий фильмов 
data['length'] = data['original_title'].str.len()

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [122]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 

data[data.budget == data.budget.max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

In [123]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'

In [0]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [125]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals'

In [124]:
data[data.runtime == data.runtime.max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [126]:
data[data.runtime == data.runtime.min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

In [127]:
answers['3'] = 'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [128]:
data.runtime.mean()

109.6585494970884

In [129]:
answers['4'] = '109.6585494970884'

# 5. Каково медианное значение длительности фильмов? 

In [130]:
data.runtime.median()

107.0

In [131]:
answers['5'] ='107.0'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [132]:
data[data.profit == data.profit.max()].original_title

239    Avatar
Name: original_title, dtype: object

In [133]:
answers['6'] ='Avatar'

# 7. Какой фильм самый убыточный? 

In [134]:
data[data.profit == data.profit.min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

In [135]:
answers['7']='The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [136]:
data[data.revenue > data.budget]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,timestamp,month,ones,tagline_length
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810,2015-06-09,6,1,17
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015,228436354,2015-05-13,5,1,18
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015,185238201,2015-03-18,3,1,26
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015,1868178225,2015-12-15,12,1,29
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015,1316249360,2015-04-01,4,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,tt0171433,30000000,37036004,Keeping the Faith,Ben Stiller|Edward Norton|Jenna Elfman|Anne Ba...,Edward Norton,"If you have to believe in something, you may a...","Best friends since they were kids, Rabbi Jacob...",127,Comedy,Spyglass Entertainment|Touchstone Pictures|Tri...,4/14/2000,5.7,2000,7036004,2000-04-14,4,1,69
1875,tt0219854,65000000,69700000,The Kid,Bruce Willis|Spencer Breslin|Emily Mortimer|Li...,Jon Turteltaub,Nobody ever grows up quite like they imagined.,Powerful businessman Russ Duritz is self-absor...,104,Fantasy|Comedy|Family,Walt Disney Pictures,7/7/2000,5.9,2000,4700000,2000-07-07,7,1,46
1877,tt0199725,20000000,27728118,Love & Basketball,"Chris Warren, Jr.|Kyla Pratt|Sanaa Lathan|Omar...",Gina Prince-Bythewood,All's fair in love and basketball.,A young African-American couple navigates the ...,124,Action|Comedy|Drama|Romance,40 Acres & A Mule Filmworks,4/21/2000,7.3,2000,7728118,2000-04-21,4,1,34
1883,tt0122459,24000000,36609995,Return to Me,David Duchovny|Minnie Driver|Carroll O'Connor|...,Bonnie Hunt,A comedy straight from the heart,It took a lot of cajoling to get Bob (Duchovny...,115,Romance|Comedy|Drama,Metro-Goldwyn-Mayer (MGM)|JLT Productions,4/7/2000,5.5,2000,12609995,2000-04-07,4,1,32


In [137]:
answers['8']='1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [391]:
data_sorted = data.query('release_year == 2008')
data_sorted[
            data_sorted.revenue == 
            data_sorted.revenue.max()
           ].original_title


599    The Dark Knight
Name: original_title, dtype: object

In [139]:
answers['9']='The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [141]:
data_sorted = data.query('release_year >= 2012 & release_year <= 2014')

In [144]:
data_sorted[data_sorted.profit == data_sorted.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,timestamp,month,ones,tagline_length
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090,2013-07-03,7,1,23


In [145]:
answers['10']='The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [177]:
#genres_ds= data.genres


#for i in range(0,len(genres_ds)):
 #   genres_ds.iindex[i]=str(i)
#pd.DataFrame(mlb.fit_transform(genres_ds,columns=mlb.classes_,index=genres_ds.index))
#genres_ds


ВАРИАНТ 2

In [181]:
genres_extracted.apply('sum').sort_values(ascending=False)

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

In [305]:
answers['11']='Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [206]:
genres_is_profit = genres_extracted.join(data['isprofit'])

In [393]:
genres_is_profit.groupby(
                         'isprofit').agg(
                         'sum').T.rename_axis(
                         'isprofit').sort_values(
                         1,ascending=False
                        )

isprofit,0,1
isprofit,,
Drama,222,560
Comedy,132,551
Thriller,150,446
Action,138,444
Adventure,78,337
Romance,66,242
Crime,84,231
Family,34,226
Science Fiction,53,195


In [225]:
answers['12']='Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [226]:
max_revenue = data[['director','revenue']].groupby('director')
max_revenue = max_revenue.agg('sum')
max_revenue = max_revenue.sort_values(by='revenue',ascending=False)

,revenue
director,
Peter Jackson,6490593685
Christopher Nolan,4167548502
David Yates,4154295625
Michael Bay,3886938960
J.J. Abrams,3579169916
...,...
David MichÃ´d,2295423
Steven Shainberg,2281089
Paul Schrader,2062066


In [227]:
answers['13']='Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [387]:
director = pd.DataFrame(data['director']).join(genres_extracted['Action'])
director = director.groupby('director').agg('sum')
director = director.sort_values(by='Action',ascending=False).head(40)
director

,Action
director,
Michael Bay,7
Paul W.S. Anderson,7
Robert Rodriguez,6
Antoine Fuqua,6
Ridley Scott,6
Peter Jackson,5
Peter Berg,5
Tony Scott,5
Brett Ratner,5


In [231]:
answers['14']='Michael Bay'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [275]:
sorted_data=data[['release_year','revenue','castlist']]


In [279]:
sorted_data = sorted_data[sorted_data.release_year == 2012]
sorted_data = sorted_data.explode('castlist').groupby('castlist')
sorted_data = sorted_data.agg('sum').sort_values('revenue',ascending=False)

,release_year,revenue
castlist,,
Chris Hemsworth,8048,2027450773
Denis Leary,4024,1629460639
Anne Hathaway,4024,1522851057
Robert Downey Jr.,2012,1519557910
Mark Ruffalo,2012,1519557910
...,...,...
Michael Nyqvist,2012,3428048
Danny Huston,2012,2106557
Josh Lucas,2012,2106557


In [285]:
answers['15']='Chris Hemsworth'

In [268]:
sorted_data.drop('cast',axis=1)

,release_year,revenue
0,2015,1513528810
1,2015,378436354
2,2015,295238201
3,2015,2068178225
4,2015,1506249360
...,...,...
1884,2000,157299717
1885,2000,13555988
1886,2000,35134820
1887,2000,36037909


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [286]:

answers['16']='Matt Damon'


In [316]:
#создадим датафрейм, в котором будут актёры, снявшиеся в фильмах
#бюджет которых больше среднего бюджета по всему датасету

In [405]:
sorted_data = cast_extracted.join(data.budget)
sorted_data = sorted_data.melt(id_vars='budget', value_name='cast')
sorted_data = sorted_data[['cast','budget']]
sorted_data['ones']= 1

over_mean_budget = sorted_data[sorted_data.budget>data.budget.mean()]
over_mean_budget = over_mean_budget.groupby('cast').agg('sum')
over_mean_budget = over_mean_budget.sort_values('ones',ascending=False)

In [ ]:
over_mean_budget

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [83]:
answers['17']='Action'

In [421]:
a=data.genres.str.split('|')
b=data.cast.str.split('|')
sorted_data = pd.DataFrame([a,b]).T
sorted_data = sorted_data.explode('cast').explode('genres')


In [422]:
sorted_data

,genres,cast
0,Action,Chris Pratt
0,Adventure,Chris Pratt
0,Science Fiction,Chris Pratt
0,Thriller,Chris Pratt
0,Action,Bryce Dallas Howard
...,...,...
1888,Thriller,Susan Ward
1888,Thriller,Lori Heuring
1888,Thriller,Matthew Settle
1888,Thriller,Nathan Bexton


In [423]:
sorted_data[sorted_data.cast=='Nicolas Cage'].describe()

,genres,cast
count,93,93
unique,15,1
top,Action,Nicolas Cage
freq,17,93


# 18. Самый убыточный фильм от Paramount Pictures

In [288]:
answers['18']='K-19: The Widowmaker'

In [424]:
sorted_data = data[[
                     'original_title',
                     'production_companies', 
                     'profit'
                  ]]

In [425]:
sorted_data[
    sorted_data.production_companies.str.contains('Paramount Pictures')
           ].sort_values('profit',ascending=True)

,original_title,production_companies,profit
925,K-19: The Widowmaker,Paramount Pictures|Intermedia Films|National G...,-64831034
1123,Timeline,Paramount Pictures|Donners' Company|Mutual Fil...,-60519261
1722,Next,Paramount Pictures|Virtual Studios|Revolution ...,-51788987
1623,Alfie,Paramount Pictures|Patalex Productions,-46604061
1631,Twisted,Paramount Pictures|Kopelson Entertainment|Harl...,-24805000
...,...,...,...
730,Mission: Impossible - Ghost Protocol,Paramount Pictures|Skydance Productions|Bad Ro...,549713380
1738,Transformers,Paramount Pictures|DreamWorks SKG|Amblin Enter...,559709780
603,Indiana Jones and the Kingdom of the Crystal S...,Lucasfilm|Paramount Pictures,601636033
317,Transformers: Revenge of the Fallen,Paramount Pictures|DreamWorks SKG|Amblin Enter...,686297228


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [289]:
answers['19']='2015'

In [145]:
best_revenue = data[['release_year','revenue']].groupby('release_year')
best_revenue = best_revenue.agg('sum').sort_values('revenue',ascending=False)

,revenue
release_year,
2015,25449202382
2014,23405862953
2013,23213799791
2012,23079001687
2011,22676791872
2010,21071204961
2009,20261791024
2008,18252781990
2007,18162406801


# 20. Какой самый прибыльный год для студии Warner Bros?

In [ ]:
answers['20']='2014'

In [432]:
sorted_data = data[[
                    'release_year',
                    'revenue',
                    'companies_list'
                  ]]

In [338]:
#это особенный вопрос, так как в датасете есть две компании, содержащие Warner Bros. в названии,
#одна из них -pictures, и вместе с этими данными ответ приобретает правильность. Возможно дело просто в том, что
#компания Warner Bros. Pictures является частью студии Warner Bros., что по хорошему должно быть отражено
#в датасете, как отношение вложенности множеств.

In [433]:
sorted_data = sorted_data.explode('companies_list')

In [434]:
sorted_data = sorted_data[
    sorted_data.companies_list.str.contains('Warner Bros')
                         ]

sorted_data = sorted_data.groupby('release_year').agg('sum')
sorted_data = sorted_data.sort_values('revenue',ascending=False)

In [435]:
sorted_data

,revenue
release_year,
2014,3246549646
2007,3183675217
2008,3050595031
2010,3008712985
2011,2845393682
2003,2814008221
2009,2737954136
2004,2731933725
2013,2599953400


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [ ]:
answers['21']='9'

In [436]:
data[[
    'month',
    'ones'
    ]].groupby(
               'month'
              ).agg(
                    'sum'
                   ).sort_values('ones',ascending=False)

,ones
month,
9,227
12,190
10,186
8,161
3,156
4,149
6,147
11,146
7,142


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [ ]:
answers['22']='450'

In [244]:
data[(data.month>5) & (data.month<9)].ones.sum()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [ ]:
answers['23']='Peter Jackson'

In [437]:
sorted_data=data[[
                  'director',
                  'month',
                  'ones'
                ]]
a = sorted_data[sorted_data.month > 11]
b = sorted_data[sorted_data.month < 3]
sorted_data = pd.merge(a,b,how='outer')

In [438]:
sorted_data = sorted_data[['director','ones']].groupby('director')
sorted_data = sorted_data.agg('sum').sort_values('ones',ascending=False)

In [439]:
sorted_data

,ones
director,
Peter Jackson,7
Steven Soderbergh,6
Clint Eastwood,6
Adam Shankman,4
Martin Scorsese,4
...,...
GÃ¡bor CsupÃ³,1
Guy Ritchie,1
Guy Ferland,1


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [ ]:
answers['24']='Twentieth Century Fox Film Corporation|Four By Two Productions'

In [298]:
data.sort_values('length',ascending=False)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,vote_average,release_year,profit,timestamp,month,ones,tagline_length,isprofit,castlist,length
1448,tt0443453,18000000,261572744,Borat: Cultural Learnings of America for Make ...,Sacha Baron Cohen|Ken Davitian|Luenell|Pamela ...,Larry Charles,"Come to Kazakhstan, it's nice!",Kazakh journalist Borat Sagdiyev travels to Am...,82,Comedy,...,6.4,2006,243572744,2006-11-02,11,1,30,1,"[Sacha Baron Cohen, Ken Davitian, Luenell, Pam...",83
1314,tt0363771,180000000,748806957,"The Chronicles of Narnia: The Lion, the Witch ...",William Moseley|Anna Popplewell|Skandar Keynes...,Andrew Adamson,Evil Has Reigned For 100 Years...,"Siblings Lucy, Edmund, Susan and Peter step th...",143,Adventure|Family|Fantasy,...,6.5,2005,568806957,2005-12-07,12,1,33,1,"[William Moseley, Anna Popplewell, Skandar Key...",62
219,tt1698641,28000000,100654149,"Alexander and the Terrible, Horrible, No Good,...",Steve Carell|Jennifer Garner|Ed Oxenbould|Dyla...,Miguel Arteta,One day can change everything.,Alexander's day begins with gum stuck in his h...,81,Family|Comedy,...,6.2,2014,72654149,2014-10-08,10,1,30,1,"[Steve Carell, Jennifer Garner, Ed Oxenbould, ...",59
1751,tt0443680,30000000,14711793,The Assassination of Jesse James by the Coward...,Brad Pitt|Casey Affleck|Sam Shepard|Mary-Louis...,Andrew Dominik,Beyond the myth lies America's greatest betrayal,Outlaw Jesse James is rumored be the 'fastest ...,160,Action|Drama|Western,...,6.9,2007,-15288207,2007-09-02,9,1,48,0,"[Brad Pitt, Casey Affleck, Sam Shepard, Mary-L...",58
385,tt0980970,155000000,415686217,The Chronicles of Narnia: The Voyage of the Da...,Skandar Keynes|Georgie Henley|Simon Pegg|Gary ...,Michael Apted,Return to magic. Return to hope. Return to Nar...,"This time around Edmund and Lucy Pevensie, alo...",113,Adventure|Family|Fantasy,...,6.0,2010,260686217,2010-08-13,8,1,50,1,"[Skandar Keynes, Georgie Henley, Simon Pegg, G...",56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,tt0449089,50000000,87528173,RV,Robin Williams|Cheryl Hines|Joanna 'JoJo' Leve...,Barry Sonnenfeld,One Family. Eight Wheels. No brakes.,Climbing aboard their mammoth recreational veh...,99,Comedy|Family|Adventure,...,5.7,2006,37528173,2006-04-28,4,1,36,1,"[Robin Williams, Cheryl Hines, Joanna 'JoJo' L...",2
714,tt1175491,25100000,29506464,W.,Josh Brolin|Elizabeth Banks|Ellen Burstyn|Ioan...,Oliver Stone,A life misunderestimated.,"Whether you love him or hate him, there is no ...",131,Drama|History,...,5.6,2008,4406464,2008-10-17,10,1,25,1,"[Josh Brolin, Elizabeth Banks, Ellen Burstyn, ...",2
1172,tt0290334,110000000,214948780,X2,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,The time has come for those who are different ...,Professor Charles Xavier and his team of genet...,133,Adventure|Action|Science Fiction|Thriller,...,6.7,2003,104948780,2003-04-24,4,1,62,1,"[Patrick Stewart, Hugh Jackman, Ian McKellen, ...",2
1282,tt0453562,40000000,95020213,42,Chadwick Boseman|T.R. Knight|Harrison Ford|Nic...,Brian Helgeland,The True Story Of An American Legend,"The powerful story of Jackie Robinson, the leg...",128,Drama,...,6.8,2013,55020213,2013-04-12,4,1,36,1,"[Chadwick Boseman, T.R. Knight, Harrison Ford,...",2


In [299]:
data.production_companies[1448]

'Twentieth Century Fox Film Corporation|Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [455]:
answers['25']='Midnight Picture Show'

In [451]:
words = data['overview'].str.split()
data['words_counted'] = words.str.len()
sorted_data = data[['companies_list','words_counted']]

In [452]:
sorted_data = sorted_data.explode('companies_list')
sorted_data = sorted_data.groupby('companies_list')
sorted_data = sorted_data.agg('mean')
sorted_data = sorted_data.sort_values('words_counted',ascending=False)

In [453]:
sorted_data

,words_counted
companies_list,
Midnight Picture Show,175.0
Room 9 Entertainment,161.0
Heineken Branded Entertainment,159.0
98 MPH Productions,159.0
Brookwell-McNamara Entertainment,156.0
...,...
Henceforth,13.0
Our Stories Films,13.0
London Boulevard,13.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [ ]:
answers['26']='The Dark Knight'

In [28]:
data.vote_average.describe()

count    1889.000000
mean        6.140762
std         0.764763
min         3.300000
25%         5.600000
50%         6.100000
75%         6.600000
max         8.100000
Name: vote_average, dtype: float64

In [458]:
8.1 - 8.1/100

8.019

In [459]:
8.1-((8.1-3.3)/100)

8.052

In [ ]:
#единственный вопрос, ответ на который не получается воспроизвести. Что считать одним процентом лучших фильмов?
#Если считать одним процентом один процент от лучшей оценки, приняв её за 100%, то мы имеем интервал
#от 8.019 до 8.1, в котором находится только один фильм - The Dark Knight.
#Если считать интервал от 0 до 100 % интервалом от 3.3 до 8.1, то в таком случае в один процент
#лучших фильмов попадут фильмы с рейтингами от 8.052 до 8.1, и это по-прежнему 1 фильм, о котором было сказано выше.


In [380]:
data[['original_title','vote_average']].sort_values('vote_average',ascending=False).head(30)

,original_title,vote_average
599,The Dark Knight,8.1
118,Interstellar,8.0
125,The Imitation Game,8.0
9,Inside Out,8.0
34,Room,8.0
1183,The Wolf of Wall Street,7.9
128,Gone Girl,7.9
1191,12 Years a Slave,7.9
119,Guardians of the Galaxy,7.9
1081,The Lord of the Rings: The Return of the King,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [ ]:
#Чтобы ответить на данный вопрос, нужно  актеров в каждом фильме сортировать по алфавиту
#А затем создать все возможные пары сочетаний этих актеров. Сортировка нужна для того,
#чтобы пара актер1,актер2 не отличалась от пары актер2, актер1.

In [472]:
cast = data.cast.str.split('|')

In [47]:
answers['27']=['Daniel Radcliffe,Rupert Grint','Emma Watson,Rupert Grint','Daniel Radcliffe,Emma Watson']

In [473]:
def list_pairs(lis):
    
    empty_list=[]
    for i in list(itertools.combinations(range(len(lis)), 2)):
        pair=[]
        for j in i:
            pair.append(lis[j])
        empty_list.append(pair)
    return empty_list    

In [474]:
sorted_cast = pd.Series(dtype='object')

for i in range(0,len(cast)):
    a = list_pairs(cast[i])
    b = pd.Series(a,dtype='object')
    sorted_cast = sorted_cast.append(b)

sorted_cast.name = 'cast'
sorted_cast = pd.DataFrame(sorted_cast)
sorted_cast['ones'] = 1
sorted_cast['pairs'] = [','.join(map(str, l)) for l in sorted_cast['cast']]

In [475]:
sorted_cast.groupby('pairs').agg('sum').sort_values('ones',ascending=False).head(20)

,ones
pairs,
"Daniel Radcliffe,Rupert Grint",8
"Daniel Radcliffe,Emma Watson",8
"Rupert Grint,Emma Watson",7
"Ben Stiller,Owen Wilson",6
"Johnny Depp,Helena Bonham Carter",6
"Robert Pattinson,Taylor Lautner",5
"Kristen Stewart,Taylor Lautner",5
"Hugh Jackman,Ian McKellen",5
"Kristen Stewart,Robert Pattinson",5


# Submission

In [306]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'26': 'The Dark Knight',
 '28': 'The Dark Knight',
 '1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': '109.6585494970884',
 '5': '107.0',
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': '1478',
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Michael Bay',
 '27': ['Daniel Radcliffe,Rupert Grint',
  'Emma Watson,Rupert Grint',
  'Daniel Radcliffe,Emma Watson'],
 '11': 'Drama',
 '15': 'Chris Hemsworth',
 '16': 'Tom Cruise',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2007',
 '21': '9',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Twentieth Century Fox Film Corporation|Four By Two Productions',
 '25': ('Columbia Pictures',
  'Clinica Estetico',
  'Propaganda Films',
  'Beverly Detroit',
  'Intermedia')}

In [70]:
# и убедиться что ни чего не пропустил)
len(answers)

1